## 5.4 潜在変数モデル

### 共通処理

In [ ]:
%matplotlib inline
# 日本語化ライブラリ導入
!pip install japanize-matplotlib | tail -n 1

In [ ]:
# ライブラリのimport

# NumPy用ライブラリ
import numpy as np

# Matplotlib中のpyplotライブラリのインポート
import matplotlib.pyplot as plt

# matplotlib日本語化対応ライブラリのインポート
import japanize_matplotlib

# pandas用ライブラリ
import pandas as pd

# データフレーム表示用関数
from IPython.display import display

# seaborn
import seaborn as sns

# 表示オプション調整

# NumPy表示形式の設定
np.set_printoptions(precision=3, floatmode='fixed')

# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14

# サイズ設定
plt.rcParams['figure.figsize'] = (6, 6)

# 方眼表示ON
plt.rcParams['axes.grid'] = True

# データフレームでの表示精度
pd.options.display.float_format = '{:.3f}'.format

# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)

In [ ]:
import pymc as pm
import arviz as az

print(f"Running on PyMC v{pm.__version__}")
print(f"Running on ArViz v{az.__version__}")

### 5.4.1 問題設定
アイリスデータセットで特定項目の値のみを利用する。  
花の種別の情報をなしで、2種類の花の統計的特徴を推論する。


### 5.4.2 データ準備

#### データ読み込みと確認

In [ ]:
# アイリスデータセットの読み込み
df = sns.load_dataset('iris')

# 先頭5行の確認
display(df.head())

#  speciesの分布確認
df['species'].value_counts()

#### 分析対象データの絞り込み


In [ ]:
# 花の種類をsetosa以外の２種類に絞り込む
df2 = df.query('species != "setosa"')

# インデックスを0から振り直す
df2 = df2.reset_index(drop=True)

# petal_widthの項目値をx_dataにセット
X = df2['petal_width'].values

#### 分析対象データを色分けなしにヒストグラム表示

In [ ]:
bins = np.arange(0.8, 3.0, 0.1)
fig, ax = plt.subplots()
sns.histplot(bins=bins, x=X, ax=ax)
ax.xaxis.set_tick_params(rotation=90)
ax.set_title('petal_widthのヒストグラム')
ax.set_xticks(bins);

####  petal_widthのヒストグラム描画(花の種類で色分け)

In [ ]:
bins = np.arange(0.8, 3.0, 0.1)
fig, ax = plt.subplots()
sns.histplot(data=df2, bins=bins, x='petal_width',
    hue='species', kde=True)
ax.xaxis.set_tick_params(rotation=90)
ax.set_title('petal_widthのヒストグラム')
ax.set_xticks(bins);

### 5.4.3 確率モデル定義

#### 潜在変数モデルの確率モデル定義

In [ ]:
#  変数の初期設定

# 何種類の正規分布モデルがあるか
n_components = 2

# 観測データ件数
N = X.shape

model1 = pm.Model()

with model1:
    # Xの観測値をConstantDataとして定義
    X_data = pm.ConstantData('X_data', X)

    # p: 潜在変数が1の値を取る確率
    p = pm.Uniform('p', lower=0.0, upper=1.0)

    # s: 潜在変数　pの確率値を基に0, 1のいずれかの値を返す
    s = pm.Bernoulli('s', p=p, shape=N)

    # mus: ２つの花の種類毎の平均値
    mus = pm.Normal('mus', mu=0.0, sigma=10.0, shape=n_components)

    # taus: 2つの花の種類毎のバラツキ
    # 標準偏差sigmasとの間には　taus = 1/(sigmas*sigmas) の関係がある
    taus = pm.HalfNormal('taus', sigma=10.0, shape=n_components)

    # グラフ描画など分析でsigmasが必要なため、tausからsigmasを求めておく
    sigmas = pm.Deterministic('sigmas', 1/pm.math.sqrt(taus))

    # 各観測値ごとに潜在変数からmuとtauを求める
    mu = pm.Deterministic('mu', mus[s])
    tau = pm.Deterministic('tau', taus[s])

    # 正規分布に従う確率変数X_obsの定義
    X_obs = pm.Normal('X_obs', mu=mu, tau=tau, observed=X_data)

#### 確率モデル構造定義

In [ ]:
g = pm.model_to_graphviz(model1)
display(g)

### 5.4.4 サンプリングと結果分析

#### サンプリング

In [ ]:
with model1:
    idata1 = pm.sample(chains=1, draws=2000, target_accept=0.99,
      random_seed=42)

#### plot_trace関数で推論結果の確認

In [ ]:
az.plot_trace(idata1, var_names=['p', 'mus', 'sigmas'], compact=False)
plt.tight_layout();

#### 各確率変数の事後分布の確認

In [ ]:
plt.rcParams['figure.figsize']=(6,6)
az.plot_posterior(idata1, var_names=['p', 'mus', 'sigmas'])
plt.tight_layout();

#### summary関数で推論結果の確認

In [ ]:
summary1 = az.summary(idata1, var_names=['p', 'mus', 'sigmas'])
display(summary1)

In [ ]:
df2.species.value_counts()

In [ ]:
df3 = df2.copy()
df3 = df3.query('species=="virginica"')
print(len(df3))

### 5.4.5 ヒストグラムと正規分布関数の重ね描き

In [ ]:
# 正規分布関数の定義
def norm(x, mu, sigma):
    return np.exp(-((x - mu)/sigma)**2/2) / (np.sqrt(2 * np.pi) * sigma)

# 推論結果から各パラメータの平均値を取得
mean = summary1['mean']

# muの平均値取得
mean_mu0 = mean['mus[0]']
mean_mu1 = mean['mus[1]']

# sigmaの平均値取得
mean_sigma0 = mean['sigmas[0]']
mean_sigma1 = mean['sigmas[1]']

# 正規分布関数値の計算
y0 = norm(x, mean_mu0, mean_sigma0) * delta / n_components
y1 = norm(x, mean_mu1, mean_sigma1) * delta / n_components

# グラフ描画
delta = 0.1
bins = np.arange(0.8, 3.0, delta)
x = np.arange(0.8, 3.0, 0.05)
plt.rcParams['figure.figsize']=(6,6)
fig, ax = plt.subplots()
sns.histplot(data=df2, bins=bins, x='petal_width',
    hue='species', kde=True, ax=ax,  stat='probability')
ax.get_lines()[0].set_label('KDE versicolor')
ax.get_lines()[1].set_label('KDE virginica')
ax.plot(x, y0, c='b', lw=3, label='Bayse versicolor')
ax.plot(x, y1, c='y', lw=3, label='Bayse virginica')
ax.set_xticks(bins);
ax.xaxis.set_tick_params(rotation=90)
ax.set_title('ヒストグラムと正規分布関数の重ね描き')
plt.legend();

### 5.4.6 潜在変数の確率分布

#### petal_widthの値が1.0, 1.5, 1.7, 2.0, 2.5のインデックスを調べる

In [ ]:
value_list = [1.0, 1.5, 1.7, 2.0, 2.5]

df_heads = pd.DataFrame(None)

# petal_widthの値が1.0から2.5までそれぞれの値である先頭の行を抽出
for value in value_list:

    # df2からpetal_widthの値がvalueである行のみ抽出
    w = df2.query('`petal_width`==@value', engine='python')

    # 先頭の１行を抽出し、df_headsに連結
    df_heads = pd.concat([df_heads, w.head(1)], axis=0)

# 結果確認
display(df_heads)

#### petal_widthの値の違いによる潜在変数sの確率分布の可視化

In [ ]:
# df_headsのインデックスを抽出
indexes = df_heads.index

# 潜在変数sのサンプル値から、index=7, 1, 27, 60, 50の値を抽出
sval = idata1.posterior['s'][:,:,indexes].values.reshape(-1,5).T

# それぞれのケースでヒストグラムの描画
plt.rcParams['figure.figsize']=(15,3)
vlist = [1.0, 1.5, 1.7, 2.0, 2.5]
fig, axes = plt.subplots(1, 5)
for ax, item, value, index in zip(axes, sval, vlist, indexes):
    f = pd.DataFrame(item)
    f.hist(ax=ax, bins=2)
    ax.set_ylim(0,2000)
    ax.set_title(f'value={value} index={index}')
plt.tight_layout();

 ### コラム　潜在変数モデルにおけるベイズ推論のツボ

#### 意図しない結果になる確率モデル

##### 確率モデル定義

In [ ]:
model2 = pm.Model()

with model2:
    # Xの観測値をConstantDataとして定義
    X_data = pm.ConstantData('X_data', X)

    # p: 潜在変数が1の値を取る確率
    p = pm.Uniform('p', lower=0.0, upper=1.0)

    # s: 潜在変数　pの確率値を基に0, 1のいずれかの値を返す
    s = pm.Bernoulli('s', p=p, shape=N)

    # mus: ２つの花の種類毎の平均値
    mus = pm.Normal('mus', mu=0.0, sigma=10.0, shape=n_components)

    # sigmas: 2つの花の種類毎のバラツキ
    sigmas = pm.HalfNormal('sigmas', sigma=10.0, shape=n_components)

    # 各観測値ごとに潜在変数から平均値と標準偏差を求める
    mu = pm.Deterministic('mu', mus[s])
    sigma = pm.Deterministic('sigma', sigmas[s])

    # 正規分布によりxの値を求める
    X_obs = pm.Normal('X_obs', mu=mu, sigma=sigma, observed=X_data)

# モデル構造可視化
g = pm.model_to_graphviz(model2)
display(g)

##### サンプリングとplot_trace関数呼び出し

In [ ]:
with model2:
    # サンプリング
    idata2 = pm.sample(random_seed=42, chains=1, target_accept=0.998)

# plot_trace関数で推論結果の確認
az.plot_trace(idata2, var_names=['p', 'mus', 'sigmas'], compact=False)
plt.tight_layout();

In [ ]:
summary2 = az.summary(idata2, var_names=['p', 'mus', 'sigmas'])
display(summary2)

#### ラベルスイッチが起きない確率モデル

##### 確率モデル定義

In [ ]:
#  変数の初期設定

# 何種類の正規分布モデルがあるか
n_components = 2

# 観測データ件数
N = X.shape

model3 = pm.Model()

with model3:
    # Xの観測値をConstantDataとして定義
    X_data = pm.ConstantData('X_data', X)

    # p: 潜在変数が1の値を取る確率
    p = pm.Uniform('p', lower=0.0, upper=1.0)

    # s: 潜在変数　pの確率値を基に0, 1のいずれかの値を返す
    s = pm.Bernoulli('s', p=p, shape=N)

    # mus: ２つの花の種類毎の平均値
    mu0 = pm.HalfNormal('mu0', sigma=10.0)
    delta0 = pm.HalfNormal('delta0', sigma=10.0)
    mu1 = pm.Deterministic('mu1', mu0+delta0)
    mus = pm.Deterministic('mus',pm.math.stack([mu0, mu1]))

    # taus: 2つの花の種類毎のバラツキ
    # 標準偏差sigmasとの間には　taus = 1/(sigmas*sigmas) の関係がある
    taus = pm.HalfNormal('taus', sigma=10.0, shape=n_components)

    # グラフ描画など分析でsigmasが必要なため、tausからsigmasを求めておく
    sigmas = pm.Deterministic('sigmas', 1/pm.math.sqrt(taus))

    # 各観測値ごとに潜在変数からmuとtauを求める
    mu = pm.Deterministic('mu', mus[s])
    tau = pm.Deterministic('tau', taus[s])

    # 正規分布に従う確率変数X_obsの定義
    X_obs = pm.Normal('X_obs', mu=mu, tau=tau, observed=X_data)

g = pm.model_to_graphviz(model3)
display(g)

##### サンプリングとplot_trace関数呼び出し

In [ ]:
with model3:
    # サンプリング
    idata3 = pm.sample(random_seed=42, target_accept=0.999)

# plot_trace関数で推論結果の確認
az.plot_trace(idata3, var_names=['p', 'mus', 'sigmas'], compact=False)
plt.tight_layout();

In [ ]:
summary3 = az.summary(idata3, var_names=['p', 'mus', 'sigmas'])
display(summary3)

##### グラフ描画

In [ ]:
# サイズ設定
plt.rcParams['figure.figsize'] = (6, 6)

# 正規分布関数の定義
def norm(x, mu, sigma):
    return np.exp(-((x - mu)/sigma)**2/2) / (np.sqrt(2 * np.pi) * sigma)

# 推論結果から各パラメータの平均値を取得
mean = summary3['mean']

# muの平均値取得
mean_mu0 = mean['mus[0]']
mean_mu1 = mean['mus[1]']

# sigmaの平均値取得
mean_sigma0 = mean['sigmas[0]']
mean_sigma1 = mean['sigmas[1]']

# 正規分布関数値の計算
y0 = norm(x, mean_mu0, mean_sigma0) * delta / n_components
y1 = norm(x, mean_mu1, mean_sigma1) * delta / n_components

# グラフ描画
delta = 0.1
bins = np.arange(0.8, 3.0, delta)
x = np.arange(0.8, 3.0, 0.05)
plt.rcParams['figure.figsize']=(6,6)
fig, ax = plt.subplots()
sns.histplot(data=df2, bins=bins, x='petal_width',
    hue='species', kde=True, ax=ax, stat='probability')
ax.get_lines()[0].set_label('KDE versicolor')
ax.get_lines()[1].set_label('KDE virginica')
ax.xaxis.set_tick_params(rotation=90)
ax.set_title('ヒストグラムと正規分布関数の重ね描き\n(ラベルスイッチ対策版)')
ax.set_xticks(bins);
ax.plot(x, y0, c='b', lw=3, label='Bayse versicolor')
ax.plot(x, y1, c='y', lw=3, label='Bayse virginica')
plt.legend();

#### バージョンの確認

In [ ]:
!pip install watermark | tail -n 1
%load_ext watermark
%watermark --iversions